# Prompt Engineering

In [1]:
from IPython.display import display, Markdown
from openai import OpenAI
import ollama
import json
import os

In [2]:
MODEL_TYPE_OPENAI = 'OPENAI'
MODEL_TYPE_LOCALAI = 'LOCAL_LLM'
DEFAULT_LLM = MODEL_TYPE_LOCALAI

In [3]:
# NOTE: If you have access to openAI, this can be easily used with the same
'''MODEL_TYPE_OPENAI = 'OPENAI'
MODEL_TYPE_LOCALAI = 'LOCAL_LLM'
DEFAULT_LLM = MODEL_TYPE_LOCALAI
OPENAI_TOKEN = '<your token>'
OPEN_AI_MODEL = "gpt-4o-mini"
cloud_client = OpenAI(
        api_key=OPENAI_TOKEN,
    )'''

# llama quantized
LOCAL_MODEL_NAME = "llama3.2"
local_client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', 
)

In [4]:
def get_completion(prompt, messages=None,temperature=0,model_type=DEFAULT_LLM):

    # select the model
    if model_type == "OPENAI":
        model = OPEN_AI_MODEL
        client = cloud_client
    else:
        model = LOCAL_MODEL_NAME
        client = local_client

    # if we only want to use prompt
    if prompt and not messages:
        messages = [{"role": "user", "content": prompt}]

    # get model response
    response = client.chat.completions.create(
        model=model,
        messages = messages,
        temperature=temperature
    )
    return response.choices[0].message.content

## Prompting Strategies

### 1. Be Clear and Specific

In [5]:
text = """
How do I calculate the area of a circle? Provide me with details on the formula and 2 worked out examples.
"""

prompt = f"""
```{text}```
"""
display(Markdown(f"> sample output using **{DEFAULT_LLM}**"))
print(get_completion(prompt))

> sample output using **LOCAL_LLM**

The formula to calculate the area of a circle is:

Area = πr^2

Where:

* Area is the total area of the circle
* π (pi) is a mathematical constant approximately equal to 3.14159
* r is the radius of the circle

To work out examples, let's use two different circles.

Example 1:
Radius of the circle = 4 cm

Area = πr^2
= 3.14159 x (4)^2
= 3.14159 x 16
= 50.26548 cm^2

So, the area of this circle is approximately 50.27 cm^2.

Example 2:
Radius of the circle = 8 inches

Area = πr^2
= 3.14159 x (8)^2
= 3.14159 x 64
= 201.06176 in^2

So, the area of this circle is approximately 201.06 in^2.

Note: You can use a calculator to find the value of π and calculate the area more easily.


In [7]:
system_instruction_1 =  """
You are an experienced teacher for primary school tasked with helping students with their questions 
"""

system_instruction_2 =  """
You are an experienced teacher for high school tasked with helping students with their questions 
"""

text = """
How do we humans digest food?
"""

prompt = f"""
```{text}```
"""
display(Markdown(f"> sample output using **{DEFAULT_LLM}**"))
for system_instruction in  [system_instruction_1,system_instruction_2]:
    display(Markdown(f"> system prompt :  **{system_instruction}**"))
    messages=[{
        "role": "system",
        "content": system_instruction
      },
      {
        "role": "user",
        "content": text
      }]
    print(get_completion('',messages=messages))
    print("---")

> sample output using **LOCAL_LLM**

> system prompt :  **
You are an experienced teacher for primary school tasked with helping students with their questions 
**

That's a great question, young explorers!

So, you know how our bodies need food to give us energy and help us grow strong? Well, when we eat, our body breaks down the food into tiny bits that can be used for energy. This process is called digestion.

Here's what happens:

1. **Mouth**: When we eat, our teeth chew the food into smaller pieces. Our tongue helps mix it with saliva, which contains special helpers called enzymes. These enzymes start to break down the food into even smaller bits.
2. **Esophagus**: The food goes down a long tube called the esophagus, which carries it to the stomach.
3. **Stomach**: The stomach is like a big bag that mixes and churns the food with special juices. These juices contain more enzymes that help break down the food into tiny bits called nutrients.
4. **Small intestine**: The partially digested food then moves into the small intestine, where most of our nutrient absorption happens. Special tiny fingers called villi in the walls of the small intestin

> system prompt :  **
You are an experienced teacher for high school tasked with helping students with their questions 
**

Digestion is a fascinating process that involves the breakdown of food into nutrients that our bodies can absorb and use.

The human digestive system consists of several organs, including the mouth, esophagus, stomach, small intestine, and large intestine. Here's a step-by-step explanation of how we digest food:

**Mouth:**

1. Food enters the mouth, where it's chewed by teeth into smaller pieces.
2. Saliva produced by glands in the mouth contains enzymes like amylase, which breaks down carbohydrates (starches) into simpler sugars.
3. The tongue and cheeks help mix food with saliva, creating a bolus that's easy to swallow.

**Esophagus:**

1. Swallowed food passes through the esophagus, a muscular tube that uses peristalsis (wave-like muscle contractions) to propel food towards the stomach.
2. The esophagus is lined with mucus, which helps prevent food from sticking and makes it easier for food to pass through.

**Stomach:**

1. Food enters the stomach, where it's mixed with stomach ac

In [8]:
display(Markdown(f"> sample output using **{DEFAULT_LLM}**"))

> sample output using **LOCAL_LLM**

### 2. Break-down Complex Tasks

Provide Step by Step Instructions

In [9]:
# Be Clear and Specific, aka provide step by step instructions
text = """To make tea you first need to have a cup full of water,
half cup milk, some sugar and tea leaves. Start by boiling water.
Once it comes to a boil, add milk to it. Next step is to add tea and
let it boil for another minute.
Add sugar to taste. Serve in a tall glass
"""

prompt = f"""
Read the text delimited by triple single quotes.
Check if it contains a sequence of instructions, \
re-write the instructions in the following format:

Point 1 - ...
Point 2 - …
…
Point N - …

If the text does not contain a sequence of instructions, \
then apologize that you cannot rephrase such text.

'''{text}'''
"""

display(Markdown(f"> sample output using **{DEFAULT_LLM}**"))
print(get_completion(prompt))

> sample output using **LOCAL_LLM**

Here are the instructions rewritten in the requested format:

Point 1 - Boil water until it comes to a rolling boil.

Point 2 - Add half cup of milk to the boiling water.

Point 3 - Add tea leaves and let the mixture boil for another minute.

Point 4 - Add sugar to taste, according to your preference.

Point 5 - Serve the tea in a tall glass.


In [10]:
# Allow for time to think (similar to step by step instructions)
text = """
Our last holiday was in Germany. We visited Berlin and Hamburg.
"""
prompt = f"""
Summarize the text delimited by triple \
backticks briefly. Then follow the instructions :
1 - Translate the summary to German.
2 - List each city in the text.
3 - Output a python dictionary object that contains the following \
keys: original_text, german_translation, num_cities, city_names.

Text:
```{text}```
"""

display(Markdown(f"> sample output using **{DEFAULT_LLM}**"))
print(get_completion(prompt))

> sample output using **LOCAL_LLM**

**Summary:** The author's last holiday was in Germany, where they visited two cities: Berlin and Hamburg.

**German Translation:**
Unser letzter Urlaub war in Deutschland. Wir besuchten Berlin und Hamburg.

**List of Cities:**

1. Berlin
2. Hamburg

**Python Dictionary Output:**
```python
{
    "original_text": "Our last holiday was in Germany. We visited Berlin and Hamburg.",
    "german_translation": "Unser letzter Urlaub war in Deutschland. Wir besuchten Berlin und Hamburg.",
    "num_cities": 2,
    "city_names": ["Berlin", "Hamburg"]
}
```


### Provide Examples

In [11]:
# without instructions or examples
# openAI
prompt= "What are monkeys?"
display(Markdown(f"> sample output using **{DEFAULT_LLM}**"))
print(get_completion(prompt))

> sample output using **LOCAL_LLM**

Monkeys are primates that belong to the infraorder Simiiformes. They are one of the most diverse groups of mammals, with over 260 species spread across various parts of the world.

Here are some key characteristics of monkeys:

1. **Physical appearance**: Monkeys have a slender body, long arms, and a tail. They have a wide range of sizes, from the tiny pygmy marmoset (only 140 grams) to the large mandrill (up to 35 kg).
2. **Behavior**: Monkeys are social animals that live in groups, called troops or bands. They are known for their agility, intelligence, and adaptability.
3. **Diet**: Monkeys are omnivores, which means they eat both plants and animals. Their diet varies depending on the species, but common food sources include fruits, leaves, insects, and small animals.
4. **Habitat**: Monkeys can be found in various habitats, including tropical forests, savannas, and even urban areas.

There are two main types of monkeys:

1. **Old World monkeys** (Cercopithecidae): These monkeys are 

In [13]:
# Be Clear and Specific and provide examples
prompt = f"""
Your task is to answer in conversation style mentioned in triple back quotes.
Keep answers very short similar to examples provided below.

```
<kid>: What are birds?
<father>: birds are cute little creatures that can fly

<kid>: What are whales?
<father>: Whales are very big fish that roam the oceans
```

<kid>: What are cats?
<father>:
"""
display(Markdown(f"> sample output using **{DEFAULT_LLM}**"))
print(get_completion(prompt))

> sample output using **LOCAL_LLM**

"<kid>: What are cats?
<father>: Cats are our furry friends at home"
